# 实验查看和管理

用于查看、对比和管理所有实验的结果。

In [ ]:
import sys
sys.path.append('src')

from src.utils import ExperimentManager
import json
import pandas as pd

# 初始化实验管理器
manager = ExperimentManager(base_dir="./experiments")

# 列出所有实验
experiments = manager.list_experiments()
print(f"找到 {len(experiments)} 个实验:")
for exp in experiments:
    print(f"  - {exp}")

## 查看单个实验详情

In [ ]:
# 查看特定实验的详细信息
experiment_name = experiments[0] if experiments else None  # 选择第一个实验，或手动指定
if experiment_name:
    info = manager.get_experiment_info(experiment_name)
    if info:
        print(f"实验: {experiment_name}")
        print(f"方法: {info['config'].get('method', 'Unknown')}")
        print(f"开始时间: {info['config'].get('start_time', 'Unknown')}")
        print(f"\n配置:")
        print(json.dumps(info['config'], indent=2, ensure_ascii=False))
        
        if info['latest_training_data']:
            train_data = info['latest_training_data']
            print(f"\n训练数据统计:")
            print(f"  - 总 episode 数: {len(train_data.get('episodes', []))}")
            if train_data.get('running_scores'):
                print(f"  - 最终运行得分: {train_data['running_scores'][-1]:.2f}")
            if train_data.get('training_records'):
                print(f"  - 最佳得分: {max(train_data['training_records']):.2f}")
        
        if info['latest_test_data']:
            test_data = info['latest_test_data']
            print(f"\n测试结果数量: {len(test_data.get('test_results', []))}")
            if test_data.get('test_results'):
                last_test = test_data['test_results'][-1]
                print(f"  最新测试结果: {json.dumps(last_test['results'], indent=2, ensure_ascii=False)}")

## 对比多个实验

In [ ]:
# 对比多个实验（例如：DANN vs Baseline）
if len(experiments) >= 2:
    # 选择要对比的实验
    exp_to_compare = experiments[:2]  # 对比前两个实验，或手动指定
    
    comparison = manager.compare_experiments(exp_to_compare)
    
    print("实验对比结果:")
    print(json.dumps(comparison, indent=2, ensure_ascii=False))
    
    # 转换为 DataFrame 便于查看
    if comparison['experiments']:
        df = pd.DataFrame(comparison['experiments'])
        print("\n对比表格:")
        print(df.to_string(index=False))
else:
    print("需要至少 2 个实验才能进行对比")

## 生成实验报告

In [ ]:
# 生成所有实验的报告
report_path = manager.generate_report("experiment_report.json")
print(f"✅ 实验报告已保存到: {report_path}")

# 读取并显示报告
with open(report_path, 'r', encoding='utf-8') as f:
    report = json.load(f)

print(f"\n总实验数: {report['total_experiments']}")
print(f"生成时间: {report['generated_at']}")
print("\n实验列表:")
for exp in report['experiments']:
    print(f"  - {exp['name']}: {exp['method']} (开始于 {exp.get('start_time', 'Unknown')})")